In [1]:
import os
os.makedirs("/root/.kaggle", exist_ok=True)
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [2]:
!kaggle datasets download ismailpromus/skin-diseases-image-dataset


Dataset URL: https://www.kaggle.com/datasets/ismailpromus/skin-diseases-image-dataset
License(s): copyright-authors
100% 5.18G/5.19G [01:31<00:00, 200MB/s]
100% 5.19G/5.19G [01:31<00:00, 60.8MB/s]


In [3]:
import zipfile
zip=zipfile.ZipFile('/content/skin-diseases-image-dataset.zip','r')
zip.extractall()
zip.close()

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Dense,Flatten,Dropout,BatchNormalization
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import cv2
import keras
import tensorflow

In [5]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau,ModelCheckpoint
from sklearn.metrics import confusion_matrix
import numpy as np


In [6]:

# -----------------------------
# 1️⃣ Data augmentation for training
# -----------------------------
train_gen = ImageDataGenerator(
    rescale=1./255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20% data for validation
)

# -----------------------------
# 2️⃣ Training data (80%)
# -----------------------------
train_data = train_gen.flow_from_directory(
    directory='/content/IMG_CLASSES',  # all images in one folder, subfolders = classes
    target_size=(160,160),
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical',
    subset='training',  # 80% for training
    shuffle=True
)

# -----------------------------
# 3️⃣ Validation data (20%)
# -----------------------------
val_data = train_gen.flow_from_directory(
    directory='/content/IMG_CLASSES',  # same folder
    target_size=(160,160),
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical',
    subset='validation',  # 20% for validation
    shuffle=False
)


Found 21726 images belonging to 10 classes.
Found 5427 images belonging to 10 classes.


In [7]:
base_model = DenseNet121(weights="imagenet", include_top=False, input_shape=(160, 160, 3))
for layer in base_model.layers:
    layer.trainable = False

model = tf.keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(64, activation='relu'),
    layers.Dense(train_data.num_classes, activation='softmax')
])

for layer in base_model.layers[-20:]:
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()
checkpoint_dir = "model_checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_path = os.path.join(checkpoint_dir, "best_model.h5")

callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7, verbose=1),
    ModelCheckpoint(filepath=checkpoint_path, monitor='val_accuracy', save_best_only=True, verbose=1)
]


history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=50,
    callbacks=callbacks,
    shuffle=True
)



29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ densenet121 (Functional)        │ (None, 5, 5, 1024)     │     7,037,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1024)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        65,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,103,754 (27.10 MB)

 Trainable params: 433,418 (1.65 MB)

...

679/679 ━━━━━━━━━━━━━━━━━━━━ 303s 447ms/step - accuracy: 0.7234 - loss: 0.7304 - val_accuracy: 0.5111 - val_loss: 1.4423 - learning_rate: 1.0000e-04
Epoch 5/50
679/679 ━━━━━━━━━━━━━━━━━━━━ 0s 351ms/step - accuracy: 0.7409 - loss: 0.6957
Epoch 5: val_accuracy did not improve from 0.51115
679/679 ━━━━━━━━━━━━━━━━━━━━ 293s 431ms/step - accuracy: 0.7408 - loss: 0.6957 - val_accuracy: 0.5003 - val_loss: 1.5341 - learning_rate: 1.0000e-04
Epoch 6/50
679/679 ━━━━━━━━━━━━━━━━━━━━ 0s 357ms/step - accuracy: 0.7598 - loss: 0.6460
Epoch 6: val_accuracy did not improve from 0.51115
679/679 ━━━━━━━━━━━━━━━━━━━━ 298s 438ms/step - accuracy: 0.7598 - loss: 0.6460 - val_accuracy: 0.4997 - val_loss: 1.5593 - learning_rate: 1.0000e-04
Epoch 7/50
679/679 ━━━━━━━━━━━━━━━━━━━━ 0s 364ms/step - accuracy: 0.7690 - loss: 0.6188
Epoch 7: val_accuracy improved from 0.51115 to 0.51612, saving model to model_checkpoints/best_model.h5


679/679 ━━━━━━━━━━━━━━━━━━━━ 301s 444ms/step - accuracy: 0.7690 - loss: 0.6188 - val_accuracy: 0.5161 - val_loss: 1.5298 - learning_rate: 1.0000e-04
Epoch 8/50
679/679 ━━━━━━━━━━━━━━━━━━━━ 0s 361ms/step - accuracy: 0.7774 - loss: 0.6007
Epoch 8: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.

Epoch 8: val_accuracy did not improve from 0.51612
679/679 ━━━━━━━━━━━━━━━━━━━━ 299s 441ms/step - accuracy: 0.7774 - loss: 0.6007 - val_accuracy: 0.5065 - val_loss: 1.6106 - learning_rate: 1.0000e-04
Epoch 9/50
679/679 ━━━━━━━━━━━━━━━━━━━━ 0s 360ms/step - accuracy: 0.7915 - loss: 0.5605
Epoch 9: val_accuracy did not improve from 0.51612
679/679 ━━━━━━━━━━━━━━━━━━━━ 298s 439ms/step - accuracy: 0.7915 - loss: 0.5605 - val_accuracy: 0.5053 - val_loss: 1.6141 - learning_rate: 5.0000e-05
Epoch 10/50
679/679 ━━━━━━━━━━━━━━━━━━━━ 0s 364ms/step - accuracy: 0.7970 - loss: 0.5498
Epoch 10: val_accuracy improved from 0.51612 to 0.52036, saving model to model_checkpoints/best_model.h5


679/679 ━━━━━━━━━━━━━━━━━━━━ 303s 446ms/step - accuracy: 0.7970 - loss: 0.5498 - val_accuracy: 0.5204 - val_loss: 1.5762 - learning_rate: 5.0000e-05
Epoch 11/50
679/679 ━━━━━━━━━━━━━━━━━━━━ 0s 361ms/step - accuracy: 0.8047 - loss: 0.5297
Epoch 11: val_accuracy did not improve from 0.52036
679/679 ━━━━━━━━━━━━━━━━━━━━ 300s 441ms/step - accuracy: 0.8047 - loss: 0.5297 - val_accuracy: 0.5134 - val_loss: 1.6045 - learning_rate: 5.0000e-05
Epoch 12/50
679/679 ━━━━━━━━━━━━━━━━━━━━ 0s 361ms/step - accuracy: 0.8078 - loss: 0.5225
Epoch 12: val_accuracy improved from 0.52036 to 0.52699, saving model to model_checkpoints/best_model.h5


679/679 ━━━━━━━━━━━━━━━━━━━━ 322s 442ms/step - accuracy: 0.8078 - loss: 0.5225 - val_accuracy: 0.5270 - val_loss: 1.6285 - learning_rate: 5.0000e-05
Epoch 13/50
679/679 ━━━━━━━━━━━━━━━━━━━━ 0s 357ms/step - accuracy: 0.8167 - loss: 0.4964
Epoch 13: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-05.

Epoch 13: val_accuracy improved from 0.52699 to 0.54376, saving model to model_checkpoints/best_model.h5


679/679 ━━━━━━━━━━━━━━━━━━━━ 299s 440ms/step - accuracy: 0.8167 - loss: 0.4964 - val_accuracy: 0.5438 - val_loss: 1.5616 - learning_rate: 5.0000e-05
Epoch 14/50
679/679 ━━━━━━━━━━━━━━━━━━━━ 0s 365ms/step - accuracy: 0.8247 - loss: 0.4812
Epoch 14: val_accuracy did not improve from 0.54376
679/679 ━━━━━━━━━━━━━━━━━━━━ 302s 445ms/step - accuracy: 0.8247 - loss: 0.4812 - val_accuracy: 0.5314 - val_loss: 1.6114 - learning_rate: 2.5000e-05
Epoch 15/50
679/679 ━━━━━━━━━━━━━━━━━━━━ 0s 359ms/step - accuracy: 0.8263 - loss: 0.4743
Epoch 15: val_accuracy did not improve from 0.54376
679/679 ━━━━━━━━━━━━━━━━━━━━ 326s 480ms/step - accuracy: 0.8263 - loss: 0.4743 - val_accuracy: 0.5205 - val_loss: 1.6617 - learning_rate: 2.5000e-05
Epoch 16/50
679/679 ━━━━━━━━━━━━━━━━━━━━ 0s 358ms/step - accuracy: 0.8213 - loss: 0.4868
Epoch 16: val_accuracy did not improve from 0.54376
679/679 ━━━━━━━━━━━━━━━━━━━━ 298s 439ms/step - accuracy: 0.8214 - loss: 0.4868 - val_accuracy: 0.5253 - val_loss: 1.6643 - learnin